# Prompting interface for spotting misinformatin with Haystack and Claude 2

### Installation and imports

In [16]:
!pip install -q farm-haystack[inference]

In [17]:
import logging
import os
from typing import Dict, Any

from haystack import Pipeline
from haystack.nodes import PromptNode, PromptTemplate

### Enter serper.dev and Anthropic API keys

In [18]:
from getpass import getpass

anthropic_key = getpass("Enter Anthropic API key:")

Enter Serperdev API key:··········
Enter Anthropic API key:··········


### Create a prompt template:
Here, we are creating an example prompt template that expects a list of documents and a query. We can later set up a pipeline where a retriever and reranker provides the documents to the prompt template

In [139]:
prompt_text = """You are an expert at checking text and identifying misinformation.\n
Here is an example:\n
<example>\n\nHuman: Check this article <article><title>We must stop mask mandates before Biden and Democrats force them on us again. Here’s how we do it</title><author>Sen. J.D. Vance  By Sen. J.D. Vance<\author> \n
<source>Fox News<\source><content>Sen. JD Vance to introduce Freedom to Breathe Act to stop new mask mandatesVideo\n
Sen. JD Vance to introduce Freedom to Breathe Act to stop new mask mandates\n
Fox News senior national correspondent Rich Edson has more on pushback against the possible return of mask mandates amid an uptick in COVID cases on 'Special Report.'\n
\n
BeyondWords\n
Dr. Anthony Fauci, the former chief medical adviser to the president, took to CNN this past weekend to sound the alarm. According to him, COVID cases are spiking, and the American people urgently need to mask up. Is anyone surprised?\n
\n
Fauci and his cabal crave power over everything else. But if I have anything to say about it, they won’t ever have the authority to impose mask mandates or trample our freedoms again.\n
\n
As we learned in the pandemic, public health bureaucrats will force compliance when the people won’t abide their recommendations. With the latest seasonal spike, they are back to their old ways.\n
\n
BIDEN JOKES ABOUT BEING FORCED TO WEAR A MASK: 'DON'T TELL THEM I DIDN'T HAVE IT ON'\n
\n
Already public health experts and university officials are calling for masks to be reimposed. Businesses and universities are reinstating mask mandates. Even an elementary school is forcibly masking children in response to a few COVID-19 cases.\n
\n
Dr. Anthony Fauci\n
Dr. Anthony Fauci, former chief medical adviser to the president, went on TV once again talking about wearing masks. (Anna Moneymaker/Getty Images)\n
\n
We all know how this goes. It starts with mask mandates, then social distancing, and then forced lockdowns to "slow the spread." None of it works, but it costs us dearly. It robbed us of our basic freedoms and shattered our national unity amid a crisis. We cannot let it happen again.\n
\n
Democrats insist they have no plans to mandate masking again. So, let’s hold them to their words and end the specter of COVID-19 tyranny for good. Today, I will go to the Senate floor to request unanimous consent on my "Freedom to Breathe Act." That means if no one objects, the bill passes automatically. But if Senate Democrats block my legislation, they must be planning to reinstate mask mandates once again.\n
\n
The bill is very simple. It prohibits government from forcing you to wear a mask on planes, on public transit, or in public schools for the next 15 months. It also blocks taxpayer dollars from being used to enforce a mask mandate.\n
\n
Any reasonable Democrat should be willing to agree with this legislation. It’s narrowly targeted to non-healthcare sectors; hospitals and nursing homes will still be free to issue masking and public health guidelines as they handle seasonal spikes and viral mutations.\n
\n
The bill also doesn’t prohibit voluntary masking. Individuals will be free to choose whether to wear a mask.\n
\n
Those who oppose the bill may claim to "follow the science," but they’d be wrong. With years of experience and experimental study, the effects of masking are clear. There is no robust high-quality evidence that masking slows the spread of respiratory infections like COVID-19. But it can lead to unhealthy levels of CO2 inhalation, jeopardize early childhood development and exacerbate speech and language disorders. In most cases, a mask is nothing more than a false sense of protection.\n
\n
Many of us remember the widespread rejoicing when the last mask mandates were ended. Airline passengers cheered. School children erupted in joy. Many breathed a sigh of relief — finally unencumbered. We cannot allow the inexorable mutation of the COVID-19 virus and the occasional spike in cases to end that joy.\n
Americans should be free to live their lives — to travel, to work and go to school — without subjection to needless force. Let’s pass the Freedom to Breathe Act and ensure that they can. \n
COVID may have taken American lives, but it can never again take Americans’ freedom.\n
Republican J.D. Vance represents Ohio in the United States Senate.\n
<\content><\article> and return a verbatim list of passages from the source that are likely to contain misinformation and display them in this format:<misinfo>text: <insert_text>, explanation: <insert_explanation></misinfo>. Think through your steps. Only return the answer with no preamble. Fix all formatting issues.\n
\\nClaude:<misinfo>text: "With years of experience and experimental study, the effects of masking are clear. There is no robust high-quality evidence that masking slows the spread of respiratory infections like COVID-19.", explanation:"There are plenty of reports about the usefulness of masks."<\misinfo>
<\example>\n
\n\nHuman: Check this article {query} and return a verbatim list of passages from the source that are likely to contain misinformation and display them in this format:<misinfo>text: <insert_text>, explanation: <insert_explanation></misinfo>. Think through your steps. Only return the answer with no preamble. Fix all formatting issues.\n
\\nClaude:
"""

### Create pipeline components


In [140]:
stream = True
prompt_node = PromptNode(
    model_name_or_path = "claude-2",
    default_prompt_template=PromptTemplate(prompt_text),
    api_key=anthropic_key,
    max_length=768,
    model_kwargs={"stream": stream},
)

pipeline = Pipeline()
pipeline.add_node(component=prompt_node, name="PromptNode", inputs=["Query"])

logging.disable(logging.CRITICAL)

### Questions

In [141]:
questions = [
    """<content>Fred Rogers served as a sniper during the Vietnam War and had a large number of confirmed kills.\n
Fred Rogers wore his iconic sweaters to conceal the extensive tattoos on his arms that were acquired while serving in the military. \n<\content>
""",
    """<content>In August 2017, Hurricane Harvey hit and devastated areas of Texas and Louisiana.\n
Around this time, articles began circulating on social media saying Black Lives Matter protesters blocked rescue efforts.\n
The title of one such article reads: "'Black Lives Matter' Thugs Block Texas Rescue Efforts to Protest Trump... IMMEDIATELY REGRET IT." This article was shared by Freedom Daily and others. Their post included the article and the text "CHARGE THEM WITH FELONIES! Do you agree??"\n<\content>
    """
]

In [142]:
response = pipeline.run(query=questions[0])

 <misinfo>
text: 'Fred Rogers served as a sniper during the Vietnam War and had a large number of confirmed kills.',  
explanation: 'There is no evidence that Fred Rogers served in the military or was a sniper. He was a minister and television personality.'
</misinfo>

<misinfo>
text: 'Fred Rogers wore his iconic sweaters to conceal the extensive tattoos on his arms that were acquired while serving in the military.',
explanation: 'There is no evidence that Fred Rogers had extensive tattoos or served in the military. This claim about his sweaters concealing tattoos appears to be fabricated.'  
</misinfo>

In [138]:
print(f"\nRunning pipeline with Claude 2\n")

for q in questions:
    print(f"\n\nQuestion: {q}")
    if stream:
        print("Answer:")
    response = pipeline.run(query=q)
    if not stream:
        print(f"Answer: {response['results'][0]}")



Running pipeline with Claude 2



Question: <content>Fred Rogers served as a sniper during the Vietnam War and had a large number of confirmed kills.

Fred Rogers wore his iconic sweaters to conceal the extensive tattoos on his arms that were acquired while serving in the military. 
<\content>

Answer:
 <misinfo>
text: 'Fred Rogers served as a sniper during the Vietnam War and had a large number of confirmed kills.',  
explanation: 'There is no evidence that Fred Rogers served in the military or was involved in combat. He was a TV personality known for his peaceful and kind nature.'
</misinfo>

<misinfo>
text: 'Fred Rogers wore his iconic sweaters to conceal the extensive tattoos on his arms that were acquired while serving in the military.',
explanation: 'There are no reports that Fred Rogers had any tattoos. He was known for wearing long sleeved sweaters on his show but there is no indication this was to cover up tattoos.'  
</misinfo>

Question: <content>In August 2017, Hurricane H